In [6]:
'''
cp fasta file to temp, unzip it, calculate gc and delete the file
cp gff3 files to temp, unzip them, calculate intron density and delete all of them
'''
import pandas as pd
from BCBio.GFF import GFFExaminer
completeInfo = pd.read_csv("../1_Data/2_completeFileInfo.csv")

In [7]:
folderPath = "/home/richard/research/1_Data/fasta_gff_gtf_07302015" #short names
basePath = "/home/richard/research/tempFile/" #folder where files are manipulated

In [8]:
## Calculate genome gc
script = ""
for i in range(completeInfo.shape[0]):
    ## copy file
    fastaName = completeInfo.loc[i]['fastaName']
    script += "cp {}/{}/{} {}\n".format(folderPath, 
                                        completeInfo.loc[i]['shortName'], 
                                        fastaName,
                                        basePath)
    ## unzip
    script += "gunzip {}\n".format(fastaName)
    ## calculate gc
    script += 'probuild --gc --seq {} | grep -o "\S\S\.\S" >> genomeGC.csv\n'.format(fastaName[:-3])
    ## remove file
    script += "rm {}\n".format(fastaName[:-3])
    ## another empty line
    script += "\n"
with open("../tempFile/genomeGC.sh","w") as f: f.write(script)
    
# print script

# consolidate genome gc into
genomeGC = pd.read_csv("../tempFile/genomeGC.csv",header = None)
completeInfo['gc'] = genomeGC

In [9]:
## prepare for intron density calculate 
## copy, unzip files and add intron to files
script = ""
for i in range(completeInfo.shape[0]):
    gff3Name = completeInfo.loc[i]['gff3Name']
    ## copy file
    script += "cp {}/{}/{} {}\n".format(folderPath, 
                                        completeInfo.loc[i]['shortName'], 
                                        gff3Name,
                                        basePath)
    ## unzip
    script += "gunzip {}\n".format(gff3Name)
    ## add intron
    script += '/home/tool/gt/bin/gt gff3 -force -addintrons -o {} {}\n'.format("withIntron_"+gff3Name[:-3], gff3Name[:-3])
    ## another empty line
    script += "\n"
    
with open("../tempFile/gff3Annotation.sh","w") as f: f.write(script)
# print script

In [5]:
## warning: this block has huge amount of calculation
## caluclate intron density (by biopython extra package: GFFExaminer from BCBio.GFF)
num_gene = []
num_intron = []
num_exon = []

for i in range(completeInfo.shape[0]):
    
    gff3Name = completeInfo.loc[i]['gff3Name']
    
    print "{}, Working on {}.....".format(i,gff3Name)
    
    examiner = GFFExaminer()
    in_handle = open(basePath+"withIntron_"+gff3Name[:-3])
    test = examiner.available_limits(in_handle)
    
    try:
        num_gene.append(test['gff_type'][('gene',)])
    except:
        print "gene problem: ", gff3Name 
        num_gene.append(0)
        
    try:
        num_exon.append(test['gff_type'][('exon',)])
    except:
        print "exon problem: ", gff3Name 
        num_exon.append(0)
    
    try:
        num_intron.append(test['gff_type'][('intron',)])
    except:
        print "intron problem: ", gff3Name 
        num_intron.append(0)

    in_handle.close()

0, Working on Aaoar1.filtered_proteins.FilteredModels1.gff3.gz.....
1, Working on Aciri1_iso.filtered_proteins.FilteredModels1.gff3.gz.....
2, Working on Aciri1_meta.filtered_proteins.FilteredModels1.gff3.gz.....
3, Working on Acral2.filtered_proteins.FilteredModels3_1.gff3.gz.....
4, Working on Agabi_varbisH97_2.filtered_proteins.FilteredModels3.gff3.gz.....
5, Working on Agabi_varbur_1.filtered_proteins.FilteredModels1.gff3.gz.....
6, Working on Altbr1.filtered_proteins.External_gene_models.gff3.gz.....
7, Working on Amamu1.filtered_proteins.FilteredModels1.gff3.gz.....
8, Working on Amath1.filtered_proteins.FilteredModels1.gff3.gz.....
9, Working on Amore1.filtered_proteins.FilteredModels1.gff3.gz.....
10, Working on Antav1.filtered_proteins.FilteredModels1.gff3.gz.....
11, Working on Antlo1.filtered_proteins.ExternalModels.gff3.gz.....
intron problem:  Antlo1.filtered_proteins.ExternalModels.gff3.gz
12, Working on Antsi1.filtered_proteins.FilteredModels1.gff3.gz.....
13, Working on

In [15]:
## consolidate gene, exon, intron into completeInfo
completeInfo['numGene'] = num_gene
completeInfo['numExon'] = num_exon
completeInfo['numIntron'] = num_intron
completeInfo['intronDensity'] = completeInfo['numIntron']/completeInfo['numGene']
completeInfo.to_csv("../1_Data/3_completeFileInfoWithStatistics.csv", index = False)